In [8]:
import requests
import os
import json
from dotenv import load_dotenv
import pathlib
import pandas as pd
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

# name the file we want
env_file_name = 'lrhf97_twit_cred.env'

# designate the path of the file
curr_file = pathlib.Path.cwd()

# join them
env_path = curr_file.joinpath(env_file_name)

# load the file
load_dotenv(env_path)

bearer_token = os.getenv("BEARER_TOKEN")
next_token=None


def create_url():
    # Replace with user ID below - RidgeSupply =3413455348
    user_id = 3413455348
    max_results =10
    
    return "https://api.twitter.com/2/users/{}/followers?max_results={}".format(user_id,max_results)

def get_params():
    
    return {'user.fields': 'id,name,username,created_at,description,public_metrics,verified,location',
            'next_token': {}}

def bearer_oauth(r):

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FollowersLookupPython"
    return r


def connect_to_endpoint(url, params, next_token=None):
    params['next_token'] = next_token # param object recieved form the create_url function
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url = create_url()
    params = get_params()
    json_response = connect_to_endpoint(url, params,next_token)
    # print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response


ridge_followers= main()

200


In [9]:
df=pd.DataFrame.from_dict(ridge_followers['data'])
df

,username,created_at,id,verified,name,public_metrics,description,location
0,DumbMoney_AMC,2015-07-21T20:02:19.000Z,3386288339,False,J,"{'followers_count': 2, 'following_count': 74, ...","Husband, dad, cyclist and a F'n 🦧",NaN
1,frankgiraffe,2008-04-14T16:56:54.000Z,14387509,False,frankgiraffe,"{'followers_count': 215, 'following_count': 37...",Living at the intersection of geekery and the ...,"SF, CA USA"
2,jspoon13,2009-04-20T14:52:11.000Z,33535739,False,Jordan Spoon,"{'followers_count': 85, 'following_count': 606...",Slinging paints on cars for @maconisetupshop -...,"Kernersville, NC"
3,f3morganton,2021-07-22T18:55:59.000Z,1418283365577461764,False,f3morganton,"{'followers_count': 22, 'following_count': 78,...","Fitness, Fellowship, & Faith coming to Morgant...",NaN
4,CyclingDsrpt,2021-07-18T18:03:36.000Z,1416820910712135682,False,DSRPT cycling,"{'followers_count': 0, 'following_count': 47, ...",,NaN
5,boyd_steven_s,2020-06-04T13:24:01.000Z,1268533564196478987,False,Steven Boyd,"{'followers_count': 40, 'following_count': 282...","@UChicago polisci grad student - IR, formal th...",NaN
6,dandng,2009-07-02T04:08:48.000Z,52964262,False,Daniel De Nobrega,"{'followers_count': 444, 'following_count': 16...",Caraqueño,"Manchester, England"
7,tanderson999,2011-10-05T14:27:33.000Z,385446082,False,Timothy Anderson,"{'followers_count': 119, 'following_count': 75...","Rider of bikes, and drinker of beer. I like to...",NaN
8,velokicks,2017-06-22T01:09:56.000Z,877695112884633601,False,velokicks,"{'followers_count': 295, 'following_count': 23...",Have a weakness for shoes? So do we.,"Melbourne, Victoria"
9,wookieapproved,2019-08-17T22:04:22.000Z,1162847621225910272,False,Wookie,"{'followers_count': 97, 'following_count': 210...",🚴 ☕️ 🎮 🥘 📷 🪴 🥖 #adhd,"Denver, CO"


In [10]:
ridge_followers['meta']['next_token']

'43HKHO8T8HUHEZZZ'

In [11]:
import csv
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['id', 'created_at', 'location', 'description','username','name','verified', 'followers_count', 'following_count', 'tweet_count','listed_count'])
csvFile.close()

In [12]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. ID
        id = tweet['id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. location
        if ('location' in tweet):   
            location = tweet['location']
        else:
            location = " "

        # 4. description
        description = tweet['description']

        # # 5. names
        username = tweet['username']
        name = tweet['name']
        verified = tweet['verified']

        # 6. Tweet metrics
        followers_count = tweet['public_metrics']['followers_count']
        following_count = tweet['public_metrics']['following_count']
        tweet_count = tweet['public_metrics']['tweet_count']
        listed_count = tweet['public_metrics']['listed_count']

        # Assemble all data in a list
        res = [id, created_at, location, description,username,name,verified, followers_count, following_count, tweet_count,listed_count]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Followers added from this response: ", counter) 

In [13]:
append_to_csv(ridge_followers,'data.csv')

# of Followers added from this response:  10


## This is for looping over tweets - going to try to convert it to get the total list of followers

In [14]:
params

NameError: name 'params' is not defined

In [16]:

max_results = 30
max_users=10

#Total number of followers we collected from the loop
total_followers = 0

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['id', 'created_at', 'location', 'description','username','name','verified', 'followers_count', 'following_count', 'tweet_count','listed_count'])
csvFile.close()
# params['next_token']=None

for i in range(0,int(max_results/max_users)+1):

    # Inputs
    count = 0 # Counting followers per time period
    max_count = 30 # Max followers per time period
    flag = True
    # next_token = params['next_token']
    url = create_url()
    params = get_params()

    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", params['next_token'])

        # url = create_url()
        # params = get_params()
        
        json_response = connect_to_endpoint(url, params)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            params['next_token'] = json_response['meta']['next_token']
            
            print("Next Token: ", params['next_token'])
            if result_count is not None and result_count > 0 and params['next_token'] is not None:
                
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_followers += result_count
                print("Total # of followers added: ", total_followers)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_followers += result_count
                print("Total # of followers added: ", total_followers)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_followers)

-------------------
Token:  {}
200
43HKHO8T8HUHEZZZ
Next Token:  43HKHO8T8HUHEZZZ
# of Followers added from this response:  10
Total # of followers added:  10
-------------------
-------------------
Token:  43HKHO8T8HUHEZZZ
200
GOMK7O8T8HUHEZZZ
Next Token:  GOMK7O8T8HUHEZZZ
# of Followers added from this response:  10
Total # of followers added:  20
-------------------
-------------------
Token:  GOMK7O8T8HUHEZZZ
200
43HKHO8T8HUHEZZZ
Next Token:  43HKHO8T8HUHEZZZ
# of Followers added from this response:  10
Total # of followers added:  30
-------------------
-------------------
Token:  43HKHO8T8HUHEZZZ
200
GOMK7O8T8HUHEZZZ
Next Token:  GOMK7O8T8HUHEZZZ
# of Followers added from this response:  10
Total # of followers added:  40
-------------------
-------------------
Token:  GOMK7O8T8HUHEZZZ
200
GOMK7O8T8HUHEZZZ
Next Token:  GOMK7O8T8HUHEZZZ
# of Followers added from this response:  10
Total # of followers added:  50
-------------------
-------------------
Token:  {}
200
43HKHO8T8HUHEZ

KeyboardInterrupt: 